In [ ]:
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.feature_selection import (
    SelectKBest,
    f_classif,
    mutual_info_classif,
    # SelectPercentile
    ,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# blah blah

In [3]:

import numpy as np
import pandas as pd

#######################
## Datenaufbereitung ##
#######################


# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path
                                
file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows  = None

## Libraries & Settings ##
from pa_lib.file import load_bin
from pa_lib.util import cap_words
from pa_lib.log import time_log, info

import datetime as dt
from dateutil.relativedelta import relativedelta

from pa_lib.data import (
    clean_up_categoricals,
    unfactorize,
)

from pa_lib.data import desc_col

In [4]:
def load_booking_data():
    bd_raw = load_bin("vkprog\\bd_data.feather").rename(
        mapper=lambda name: cap_words(name, sep="_"), axis="columns"
    )
    bd = bd_raw.loc[(bd_raw.Netto > 0)].pipe(clean_up_categoricals)
    return bd

In [5]:
bd_raw = load_booking_data()

09:20:52 [INFO] Started loading binary file ...
09:20:52 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
09:20:52 [INFO] ... finished loading binary file in 0.18s (0.92s CPU)


In [47]:
#desc_col(ek_info)

# OMG! I FORGOT ABOUT THE IMPLICIT USE OF COLUMNS IN THE ROW SELECTION!!!

In [124]:
crm_data = load_bin("vkprog\\crm_data.feather").astype({"ENDKUNDE_NR": "int64"})

2019-10-30 14:35:36 [INFO] Started loading binary file
2019-10-30 14:35:36 [INFO] Reading from file C:\Users\stc\data\vkprog\crm_data.feather
2019-10-30 14:35:37 [INFO] Finished loading binary file in 0.21s (0.33s CPU)


In [125]:
desc_col(crm_data)

,DTYPE,NULLS,UNIQUE
BETREFF,category,26/1056374,336940
KANAL,category,0/1056400,12
STARTTERMIN,datetime64[ns],0/1056400,3817
QUELLE,category,0/1056400,3
VERANTWORTLICH,category,0/1056400,337
KUERZEL,category,1/1056399,340
ENDKUNDE_NR,int64,0/1056400,51555
VB_FILTER_VON,datetime64[ns],1054692/1708,13
VB_FILTER_BIS,datetime64[ns],1055145/1255,37
VB_FILTER_GRUND,category,1054709/1691,60


- [x] ``Kleinkunde`` (Über die letzten 4 Jahre nie mehr als 3'000 Umsatz pro Jahr)
- [x] ``Neukunde`` (Alle, die erst im aktuellen Jahr Umsatz hatten)
- [x] ``Insolvenz`` (Kunden mit Bonitäts-Flag "keine Verkäufe" werden markiert)
- [x] ``Umsatz_erreicht`` (80% Netto-Umsatz gem. Vorjahr erreicht) Die Idee war uspruenglich Kunden auszuschliessen, welche schon minimum 80% vom Vorjahresumsatz erreicht haben.
- [x] ``kuerzlich_gebucht`` (in den letzten 2 Monaten erfasste Kampagnen )
- [x] ``kuerzlich_im_aushang`` (Aushangbeginn vor 1 Monat oder später)
- [x] ``kuerzlich_im_kontakt`` (CRM-Kontakt in den letzten 4 Wochen)
- [x] ``VB_FILTER_AKTIV`` (in CRM ist eine gültige Sperre für Kunden erfasst) 

In [145]:
ek_info = load_bin("vkprog\\ek_info.feather")

2019-10-30 15:05:24 [INFO] Started loading binary file
2019-10-30 15:05:24 [INFO] Reading from file C:\Users\stc\data\vkprog\ek_info.feather
2019-10-30 15:05:24 [INFO] Finished loading binary file in 0.04s (0.03s CPU)


In [146]:
ek_info.sample(3,random_state=42)

,index,Endkunde_NR,Endkunde,EK_Aktiv,EK_Kam_Betreut,EK_Land,PLZ,GEMEINDE,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Last_Res_Date,First_Res_Year,Last_Res_Year,Last_Aus_Date,EK_HB_Apg_Kurzz,AG_Hauptbetreuer,KANTON,BEZIRK,REGION,GROSSREGION,VERKAUFS_GEBIETS_CODE,VB_VKGEB,Letzter_Kontakt,Kanal,Betreff,last_CRM_Ktkt_date,VB_FILTER_VON,VB_FILTER_BIS,Letzte_CRM_Ktkts,Net_2015,Net_2016,Net_2017,Net_2018,Net_2019,Verkaufsberater,Insolvenz
20042,20042,577920,Wizz Air Hungary Kft,1,1,HUNGARY,1185,Budapest,None,WG - Verkehrsbetriebe,410,410,1,2013-06-11,2013,2013,2013-06-17,DEA,None,None,None,None,None,INTERNATIONAL,MAT,CGE,E-Mail,Traffic SR-Actions-2016 - Offre spéciale Power...,2016-09-13,NaT,NaT,None,NaN,NaN,NaN,NaN,NaN,None,None
10311,10311,503679,Cizen Inc. sports & health,1,0,SCHWEIZ,3011,Bern,None,WG - Handel / Grossverteiler,901,"555,720,800",3,2014-10-01,2008,2014,2015-01-01,BEB,None,BE,Bern-Mittelland,Bern,Region Bern,V-M04,YAM,CJA,E-Mail,Verkauf Bern Mitte 2019 - Begleitkommunikation...,2019-09-24,NaT,NaT,"LGE,BIM,MRL,ANS,MAL,CJA",NaN,NaN,NaN,NaN,NaN,None,None
13356,13356,520701,Reist Haushaltapparate AG,1,0,SCHWEIZ,4543,Deitingen,Regio Annoncen + Verlag AG,WG - Haushaltsartikel,290,400,1,2009-11-01,2009,2009,2010-02-01,MWY,LOD,SO,Wasseramt,Solothurn,Region Biel–Jura,V-M03,RIN,None,None,None,NaT,NaT,NaT,None,NaN,NaN,NaN,NaN,NaN,None,None


In [147]:
#
desc_col(ek_info)

,DTYPE,NULLS,UNIQUE
index,int64,0/32237,32237
Endkunde_NR,int64,0/32237,32237
Endkunde,object,0/32237,30291
EK_Aktiv,int64,0/32237,2
EK_Kam_Betreut,int64,0/32237,2
EK_Land,object,49/32188,45
PLZ,int64,0/32237,2875
GEMEINDE,object,49/32188,3006
Agentur,object,20999/11238,3247
EK_BG,object,1829/30408,83


In [200]:
net_columns = [col for col in ek_info.columns if col.startswith("Net_")] # Net_2015, ..., Net_2019

row_filter = (
    # Insolvenz:
    (ek_info.loc[:,"Insolvenz"] 
         != True
    ) &
    
    # kuerzlich_gebucht (in den letzten 2 Monaten erfasste Kampagnen):
    (ek_info.loc[:,"Last_Res_Date"] 
        < pd.Timestamp.now() - pd.DateOffset(months=2)
    ) &
    
    # kuerzlich_im_aushang (Aushangbeginn vor 1 Monat oder später):
    (ek_info.loc[:,"Last_Aus_Date"] 
        < pd.Timestamp.now() - pd.DateOffset(months=1)
    ) &
    
    # keine Kleinkunden (Ueber die letzten 4 Jahre nie mehr als 3'000 pro Jahr):
    (ek_info.loc[:,net_columns].max(axis=1).fillna(0) 
        > 3000
    ) &
    
    # keine Neukunden (Alle, die erst im aktuellen Jahr Umsatz hatten):
    ((ek_info.loc[:,sorted(net_columns, reverse=True)[1:]]
             .max(axis=1)
             .fillna(0))
        > 0
    ) &
    
    # Umsatz_erreicht (80% Netto-Umsatz gem. Vorjahr erreicht) 
    ((ek_info.loc[:,sorted(net_columns, reverse=True)[0]]
             .fillna(0))
         <= 0.8*(ek_info.loc[:,sorted(net_columns, reverse=True)[1]]
                        .fillna(0))
    ) &
    
    # kuerzlich_im_kontakt (keine Kunden, mit CRM-Kontakt in den letzten 4 Wochen)
    (ek_info.loc[:,"last_CRM_Ktkt_date"].fillna(pd.Timestamp.now() - pd.DateOffset(years=100)) 
        < pd.Timestamp.now() - pd.DateOffset(months=1)
    ) &
    
    # VB_FILTER_AKTIV (in CRM ist eine gültige Sperre für Kunden erfasst)
     ~(# We define the evil ones, and take the boolean opposite:
    
         # Both entries exist: Customer is right now within "Sperre"
        ((ek_info.loc[:,"VB_FILTER_VON"] < pd.Timestamp.now()) &
         (pd.Timestamp.now() <= ek_info.loc[:,"VB_FILTER_BIS"]))

        |# No end date, but begin date exists: 
        ((ek_info.loc[:,"VB_FILTER_VON"] < pd.Timestamp.now()) &
         (ek_info.loc[:,"VB_FILTER_BIS"].isna()))

        |# No begin date, but end date exists:
        ((ek_info.loc[:,"VB_FILTER_VON"].isna() & 
         (ek_info.loc[:,"VB_FILTER_BIS"] <= pd.Timestamp.now() ))) 
      )
    
    )

display(ek_info.shape)
display(ek_info.loc[row_filter,:].shape)


(32237, 39)

(6468, 39)

In [187]:
test_row_filter = (~ek_info.loc[:,"VB_FILTER_BIS"].isna() | ~ek_info.loc[:,"VB_FILTER_VON"].isna())
container_vb_filter = ek_info.loc[test_row_filter,["Endkunde_NR", "VB_FILTER_VON","VB_FILTER_BIS"]]

In [188]:
container_vb_filter

,Endkunde_NR,VB_FILTER_VON,VB_FILTER_BIS
535,105868,2019-06-04,2040-03-31
818,109179,2019-06-03,2019-11-29
1114,113039,2019-06-25,NaT
1260,114363,2019-06-25,NaT
2248,121940,2019-05-06,2020-03-06
2783,125764,2019-07-04,NaT
2838,126259,2019-06-03,NaT
3734,134826,2019-06-03,2020-04-03
4166,138991,2019-06-03,2020-04-03
4566,147285,2019-07-16,2021-05-10


In [199]:
test_row_filter = ~(# We define the evil ones, and take the boolean opposite
    
     # Both entries exist: Customer is right now within "Sperre"
    ((ek_info.loc[:,"VB_FILTER_VON"] < pd.Timestamp.now()) &
     (pd.Timestamp.now() <= ek_info.loc[:,"VB_FILTER_BIS"]))
    
    |# No end date, but begin date exists: 
    ((ek_info.loc[:,"VB_FILTER_VON"] < pd.Timestamp.now()) &
     (ek_info.loc[:,"VB_FILTER_BIS"].isna()))
    
    |# No begin date, but end date exists:
    ((ek_info.loc[:,"VB_FILTER_VON"].isna() & 
     (ek_info.loc[:,"VB_FILTER_BIS"] <= pd.Timestamp.now() ))) 
    )

In [197]:
container_vb_filter = ek_info.loc[test_row_filter,["Endkunde_NR", "VB_FILTER_VON","VB_FILTER_BIS"]]

In [198]:
container_vb_filter

,Endkunde_NR,VB_FILTER_VON,VB_FILTER_BIS
535,105868,2019-06-04,2040-03-31
818,109179,2019-06-03,2019-11-29
1114,113039,2019-06-25,NaT
1260,114363,2019-06-25,NaT
2248,121940,2019-05-06,2020-03-06
2783,125764,2019-07-04,NaT
2838,126259,2019-06-03,NaT
3734,134826,2019-06-03,2020-04-03
4166,138991,2019-06-03,2020-04-03
4566,147285,2019-07-16,2021-05-10


In [120]:
#sticcio = sorted(net_columns, reverse=True)[1:]
#print(sticcio)

In [119]:
#ek_info.loc[:,sorted(net_columns, reverse=True)[0]].fillna(0) <= 0.8*ek_info.loc[:,sorted(net_columns, reverse=True)[1]].fillna(0) + 1

In [107]:
sorted(net_columns, reverse=True)[1]

'Net_2018'

In [95]:
print(net_columns)

['Net_2015', 'Net_2016', 'Net_2017', 'Net_2018', 'Net_2019']


In [98]:
#ek_info.loc[:,net_columns].max(axis=1).fillna(0) > 3000

In [69]:
#ek_info.loc[:,"Last_Res_Date"] > pd.Timestamp.now() - pd.DateOffset(months=2)

In [6]:
bd_raw.sample(5, random_state=42)

,Endkunde_NR,Endkunde,EK_Abc,EK_Boni,EK_Plz,EK_Ort,EK_Land,EK_HB_Apg_Kurzz,EK_Kam_Betreut,EK_Aktiv,Agentur,AG_Hauptbetreuer,Verkaufsberater,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Endkunde_Nbranchengruppe_ID,Endkunde_Nbranchengruppe,Endkunde_Branchenkat_ID,Endkunde_Branchenkat,Endkunde_Nbranchenkat_ID,Endkunde_Nbranchenkat,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe,Auftrag_Nbranchengruppe_ID,Auftrag_Nbranchengruppe,Auftrag_Branchenkat_ID,Auftrag_Branchenkat,Auftrag_Nbranchenkat_ID,Auftrag_Nbranchenkat,Agps_NR,Segment,KV_NR,KV_Typ,Kampagnen_Status,Kampagne_Erfassungsdatum,Kampagne_Beginn,Auftragsart,Res_Dat,Annullation_Datum,Aush_Von,Dauer,Vertrag,Brutto,Netto,Agglo,PF,Kamp_Beginn_Jahr,Kamp_Beginn_KW,Kamp_Beginn_KW_2,Kamp_Beginn_KW_4,Kamp_Erfass_Jahr,Kamp_Erfass_KW,Kamp_Erfass_KW_2,Kamp_Erfass_KW_4
1017991,592834,Carta turistica SA,E,gut,6901,Lugano,SCHWEIZ,SOL,0,1,NaN,NaN,ROS,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,2050838,APG|SGA,811509,KPG,4,2015-03-10,2015-06-15,Kommerziell,2015-03-27,NaT,2015-06-15,14,Nein,3733,3733,"A5192,A5250",F12,2015,25,25,25,2015,11,11,9
1197937,168605,C&A Mode AG,A,gut,6341,Baar,SCHWEIZ,DEA,1,1,"R. Koller & L. Della Piana Koller, Consulting",KUR,KUR,195,WG - Bekleidung / Wäsche,901,WG - Handel / Grossverteiler,04,WB - Bekleidung / persönlicher Bedarf,14,WB - Handel,195,WG - Bekleidung / Wäsche,901,WG - Handel / Grossverteiler,04,WB - Bekleidung / persönlicher Bedarf,14,WB - Handel,1813376,APG|SGA,754283,KPG,4,2013-11-26,2014-05-05,Kommerziell,2013-11-26,NaT,2014-05-05,14,Nein,2428,1991,"90306,90329,90404,91103,91509,92583,93293,9378...","City eBoard,City ePanel,Escalator ePanel,F12,F...",2014,19,19,17,2013,48,47,45
1137867,536344,Küry Automobile AG,D,gut,8803,Rüschlikon,SCHWEIZ,MOE,0,1,X-TRA RAINER TRUTTMANN,BGI,DSC,400,WG - Automarkt,NaN,NaN,13,WB - Verkehr,NaN,NaN,400,WG - Automarkt,NaN,NaN,13,WB - Verkehr,NaN,NaN,1768257,Traffic,742309,KDVTLG,4,2013-09-01,2013-12-01,Traffic Auftrag,2013-09-01,NaT,2013-12-01,365,Ja,7500,7500,A0261,"F12,F24,Ganzheck",2013,48,47,45,2013,35,35,33
865735,104912,APG|SGA Rail,A,gut,8027,Zürich,SCHWEIZ,YAM,0,1,NaN,NaN,JZE,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,980,WG - Spitäler / Kliniken / Kurzentren / Aerzte,NaN,NaN,21,WB - Spitäler/ Kliniken/ Kurzentren/ Aerzte,NaN,NaN,1672038,Rail,717049,KPG,4,2013-01-09,2013-01-28,Aushangauftrag Partner,2013-01-09,NaT,2013-01-28,7,Nein,576,576,"90306,90329,90404,90412,90768,91103,91362,9137...","F12,F12L,F200,F200L,F24,F4,F4C,F4S,NeonSign,Pr...",2013,5,5,5,2013,2,1,1
165668,144291,AllBlues Konzert AG,D,gut,8404,Winterthur,SCHWEIZ,ACO,0,1,NaN,NaN,ACO,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,2798438,Digital Sales,968021,KPG,4,2018-10-04,2018-10-15,Kommerziell,2018-10-04,NaT,2018-10-29,1,Nein,168,168,"A0230,A0261,A0351,A1061,A2701,A3203,A4001,A558...","City eBoard,City ePanel,Escalator ePanel,F12,F...",2018,42,41,41,2018,40,39,37


# datetime stuff


In [1]:
5+6

11

In [5]:
from pa_lib.util import iso_to_datetime

In [6]:
iso_to_datetime(2018,47,1)

datetime.datetime(2018, 11, 19, 0, 0)

In [10]:
dt.date(year=2019, month=3, day=1).isocalendar()

(2019, 9, 5)

In [2]:
import datetime as dt


In [4]:
dt.date(year=2019, month=12, day=2).isocalendar()

(2019, 49, 1)

error

In [7]:
bd_data_raw = load_bin("vkprog\\bd_data_raw.feather")

09:22:51 [INFO] Started loading binary file ...
09:22:51 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data_raw.feather
09:22:51 [INFO] ... finished loading binary file in 0.19s (0.94s CPU)


In [9]:
bd_data_raw.head().T

,0,1,2,3,4
ENDKUNDE_NR,316770,316770,119642,119642,136654
ENDKUNDE,JCDecaux Ltd.,JCDecaux Ltd.,Hublot SA,Hublot SA,Vaterländische Union
EK_ABC,NO,NO,A,A,E
EK_BONI,gut,gut,gut,gut,gut
EK_PLZ,W2 1YR,W2 1YR,1260,1260,9490
EK_ORT,London,London,Nyon 2,Nyon 2,Vaduz
EK_LAND,GREAT BRITAIN,GREAT BRITAIN,SCHWEIZ,SCHWEIZ,LIECHTENSTEIN
EK_HB_APG,16070,16070,15130,15130,351
EK_HB_APG_KURZZ,DEA,DEA,BEG,BEG,REM
EK_KAM_BETREUT,1,1,1,1,0


In [10]:
col_list = """ENDKUNDE_NR
              ENDKUNDE
              EK_ABC
              EK_BONI
              EK_PLZ
              EK_ORT
              EK_LAND
              EK_HB_APG_KURZZ
              EK_KAM_BETREUT
              EK_AKTIV
              AGENTUR
              AG_HAUPTBETREUER
              VERKAUFSBERATER
              ENDKUNDE_BRANCHENGRUPPE_ID
              ENDKUNDE_BRANCHENGRUPPE
              ENDKUNDE_NBRANCHENGRUPPE_ID
              ENDKUNDE_NBRANCHENGRUPPE
              ENDKUNDE_BRANCHENKAT_ID
              ENDKUNDE_BRANCHENKAT
              ENDKUNDE_NBRANCHENKAT_ID
              ENDKUNDE_NBRANCHENKAT
              AUFTRAG_BRANCHENGRUPPE_ID
              AUFTRAG_BRANCHENGRUPPE
              AUFTRAG_NBRANCHENGRUPPE_ID
              AUFTRAG_NBRANCHENGRUPPE
              AUFTRAG_BRANCHENKAT_ID
              AUFTRAG_BRANCHENKAT
              AUFTRAG_NBRANCHENKAT_ID
              AUFTRAG_NBRANCHENKAT
              AGPS_NR
              SEGMENT
              KV_NR 
              KV_TYP
              KAMPAGNEN_STATUS
              KAMPAGNE_ERFASSUNGSDATUM
              KAMPAGNE_BEGINN
              AUFTRAGSART
              RES_DAT
              ANNULLATION_DATUM
              AUSH_VON
              DAUER
              VERTRAG
              BRUTTO
              NETTO
              AGGLO
              PF
           """.split()

In [27]:
bd_data = (
    bd_data_raw
    # filter
    .loc[:, col_list]
    .dropna(
        how="any",
        subset="""ENDKUNDE_NR ENDKUNDE EK_AKTIV VERKAUFSBERATER AGPS_NR SEGMENT KV_NR 
                KAMPAGNEN_STATUS KAMPAGNE_ERFASSUNGSDATUM AUFTRAGSART RES_DAT AUSH_VON
                DAUER VERTRAG BRUTTO NETTO""".split(),
    )
    .query(
        "not KAMPAGNE_BEGINN < KAMPAGNE_ERFASSUNGSDATUM"
    )  # keep rows where KAMPAGNE_BEGINN is empty!
    .query("DAUER >= 0")
)

In [28]:
bd_data = (
    bd_data
    # fix ANNULLATION_DATUM < RES_DAT
    .pipe(
        replace_col,
        "ANNULLATION_DATUM",
        with_col="RES_DAT",
        where="ANNULLATION_DATUM < RES_DAT",
    )
    # fix KAMPAGNE_ERFASSUNGSDATUM < min(RES_DAT) per KV
    .pipe(calc_col_partitioned, "KV_RES_DAT", fun=np.min, on="RES_DAT", part_by="KV_NR")
    .pipe(
        replace_col,
        "KAMPAGNE_ERFASSUNGSDATUM",
        with_col="KV_RES_DAT",
        where="KAMPAGNE_ERFASSUNGSDATUM < KV_RES_DAT",
    )
    .drop("KV_RES_DAT", axis="columns")
    .fillna({"EK_KAM_BETREUT": 0})
    .pipe(clean_up_categoricals)
    .reset_index(drop=True)
)

In [15]:
from pa_lib.data import calc_col_partitioned

In [26]:
def calc_col_partitioned(df, col, fun, on, part_by):
    """Create new column 'col' in df by applying 'fun' to 'on'
       partitioned by 'part_by'"""
    new_col = df.groupby(part_by, observed=True)[on].transform(fun)
    return df.assign(**{col: new_col})


In [29]:
bd_data = (
    bd_data.pipe(
        split_date_iso,
        "KAMPAGNE_BEGINN",
        yr_col="KAMP_BEGINN_JAHR",
        kw_col="KAMP_BEGINN_KW",
    )
    .pipe(make_isoweek_rd, kw_col="KAMP_BEGINN_KW", round_by=(2, 4))
    .pipe(
        split_date_iso,
        "KAMPAGNE_ERFASSUNGSDATUM",
        yr_col="KAMP_ERFASS_JAHR",
        kw_col="KAMP_ERFASS_KW",
    )
    .pipe(make_isoweek_rd, kw_col="KAMP_ERFASS_KW", round_by=(2, 4))
)

#info(f"Cleaned data: {bd_data.shape}, size is {obj_size(bd_data)}")
store_bin(bd_data, "bd_data.feather")

09:50:02 [INFO] Started storing binary file ...
09:50:02 [INFO] Writing to file C:\Users\stc\data\bd_data.feather
09:50:03 [INFO] Written 254.4 MB
09:50:03 [INFO] ... finished storing binary file in 0.2s (1.31s CPU)


In [20]:
from pa_lib.data import (
    as_dtype,
    clean_up_categoricals,
    replace_col,
    ,
    split_date_iso,
    make_isoweek_rd,
)

In [23]:
from pa_lib.file import store_csv, store_bin, set_project_dir

In [30]:
bd_data.head()

,ENDKUNDE_NR,ENDKUNDE,EK_ABC,EK_BONI,EK_PLZ,EK_ORT,EK_LAND,EK_HB_APG_KURZZ,EK_KAM_BETREUT,EK_AKTIV,AGENTUR,AG_HAUPTBETREUER,VERKAUFSBERATER,ENDKUNDE_BRANCHENGRUPPE_ID,ENDKUNDE_BRANCHENGRUPPE,ENDKUNDE_NBRANCHENGRUPPE_ID,ENDKUNDE_NBRANCHENGRUPPE,ENDKUNDE_BRANCHENKAT_ID,ENDKUNDE_BRANCHENKAT,ENDKUNDE_NBRANCHENKAT_ID,ENDKUNDE_NBRANCHENKAT,AUFTRAG_BRANCHENGRUPPE_ID,AUFTRAG_BRANCHENGRUPPE,AUFTRAG_NBRANCHENGRUPPE_ID,AUFTRAG_NBRANCHENGRUPPE,AUFTRAG_BRANCHENKAT_ID,AUFTRAG_BRANCHENKAT,AUFTRAG_NBRANCHENKAT_ID,AUFTRAG_NBRANCHENKAT,AGPS_NR,SEGMENT,KV_NR,KV_TYP,KAMPAGNEN_STATUS,KAMPAGNE_ERFASSUNGSDATUM,KAMPAGNE_BEGINN,AUFTRAGSART,RES_DAT,ANNULLATION_DATUM,AUSH_VON,DAUER,VERTRAG,BRUTTO,NETTO,AGGLO,PF,KAMP_BEGINN_JAHR,KAMP_BEGINN_KW,KAMP_BEGINN_KW_2,KAMP_BEGINN_KW_4,KAMP_ERFASS_JAHR,KAMP_ERFASS_KW,KAMP_ERFASS_KW_2,KAMP_ERFASS_KW_4
0,316770,JCDecaux Ltd.,NO,gut,W2 1YR,London,GREAT BRITAIN,DEA,1,1,JCDecaux,MAT,MAT,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,901,WG - Handel / Grossverteiler,NaN,NaN,14,WB - Handel,NaN,NaN,1040381,APG|SGA,542212,KPG,3,2008-10-13,NaT,Kommerziell,2008-10-13,2008-11-21,2009-05-11,7,Nein,0,0,"90306,90329,90404,91103,91301,91372,91407,9150...","Banner,City eBoard,City ePanel,Cube,Escalator ...",NaN,NaN,NaN,NaN,2008,42,41,41
1,316770,JCDecaux Ltd.,NO,gut,W2 1YR,London,GREAT BRITAIN,DEA,1,1,JCDecaux,MAT,MAT,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,901,WG - Handel / Grossverteiler,NaN,NaN,14,WB - Handel,NaN,NaN,1040390,APG|SGA,542212,KPG,3,2008-10-13,NaT,Kommerziell,2008-10-13,2008-11-21,2009-05-04,7,Nein,0,0,"90306,90329,90404,91103,91301,91372,91407,9150...","Banner,City eBoard,City ePanel,Cube,Escalator ...",NaN,NaN,NaN,NaN,2008,42,41,41
2,119642,Hublot SA,A,gut,1260,Nyon 2,SCHWEIZ,BEG,1,1,NaN,NaN,CRO,555,WG - Persönlicher Bedarf,NaN,NaN,04,WB - Bekleidung / persönlicher Bedarf,NaN,NaN,555,WG - Persönlicher Bedarf,NaN,NaN,04,WB - Bekleidung / persönlicher Bedarf,NaN,NaN,1041389,APG|SGA,542499,KPG,4,2008-10-13,2009-04-27,Kommerziell,2008-10-13,NaT,2009-04-27,7,Nein,3777,3210,"96300,A0230,A0261,A0351,A0371,A1061,A2701,A293...","BIG,Caisson lumineux,EntryPortal Poster,F12,F1...",2009,18,17,17,2008,42,41,41
3,119642,Hublot SA,A,gut,1260,Nyon 2,SCHWEIZ,BEG,1,1,NaN,NaN,CRO,555,WG - Persönlicher Bedarf,NaN,NaN,04,WB - Bekleidung / persönlicher Bedarf,NaN,NaN,555,WG - Persönlicher Bedarf,NaN,NaN,04,WB - Bekleidung / persönlicher Bedarf,NaN,NaN,1048493,APG|SGA,542499,KPG,4,2008-10-13,2009-04-27,Kommerziell,2008-10-24,2008-10-27,2009-04-27,14,Nein,0,0,"96300,A0230,A0261,A0351,A0371,A1061,A2701,A293...","BIG,Caisson lumineux,EntryPortal Poster,F12,F1...",2009,18,17,17,2008,42,41,41
4,136654,Vaterländische Union,E,gut,9490,Vaduz,LIECHTENSTEIN,REM,0,1,NaN,NaN,ERS,960,WG - Parteien,NaN,NaN,19,WB - Behörden / Politik,NaN,NaN,960,WG - Parteien,NaN,NaN,19,WB - Behörden / Politik,NaN,NaN,1041827,APG|SGA,542604,KPG,4,2008-10-13,2008-12-29,Politisch,2008-10-13,NaT,2009-01-26,14,Nein,1864,1398,NaN,F12,2009,1,1,1,2008,42,41,41
